## Measuring PSFs for Multiplane Analysis

In this example we'll measure microscope PSFs for multiplane analysis (biplane in the example).

### Configuration

As with Spliner, to measure the PSF you'll need a movie of sparse fluorescent beads being scanned through the microscope focus. We typically do this with small fluorescent beads on the order of 0.1um in diameter. The beads are fixed to the coverslip using a buffer with 100mM MgCl2. The coverslip is scanned through the focus in 10nm steps using a piezo z scanner. We assume that the drift in XYZ is neglible during the time it takes to take these movies (10s of seconds).

In this example we're just going to simulate this using a distortion free theoritical PSF. 

In [ ]:
import os
os.chdir("/home/hbabcock/Data/storm_analysis/jy_testing/")
print(os.getcwd())

Generate the sample data for this example. 

Typically you will need sCMOS calibration files for your cameras as well a file containing the (first order) transform betweens the cameras.  
The `scmos_cal` notebook discusses how to calibrate a camera.  
The `micrometry_mapping` notebook discusses one approach to measuring the transform between different cameras.  

In [ ]:
import storm_analysis.jupyter_examples.multiplane_measure_psf as multiplane_measure_psf

# Make a fake CMOS calibration file.
multiplane_measure_psf.makeCMOSCalibration()

# Make a fake mapping file.
multiplane_measure_psf.makeMapping()

# Make a bead movie for each camera.
multiplane_measure_psf.makeSampleData()

### Identifying localizations to use in the PSF measurement.

The first step is to identify the localizations to use for PSF measurement. This is done by first analyzing an in focus image in channel 0 (the reference channel) with sCMOS (or 3D-DAOSTORM). Then the `psfLocalizations()` function is used to select localizations that are isolated from each other and also inside all of the images from all of the cameras.

In [ ]:
# Identify localizations using sCMOS.

# This will create an analysis XML file that we can use for sCMOS analysis.
multiplane_measure_psf.sCMOSSingleFrameXML()

# Remove stale results, if any.
if os.path.exists("c1_zcal.hdf5"):
    os.remove("c1_zcal.hdf5")
    
# Run sCMOS analysis.
import storm_analysis.sCMOS.scmos_analysis as scmosAnalysis
scmosAnalysis.analyze("c1_zcal.dax", "c1_zcal.hdf5", "scmos_single_plane.xml")

# Select good localizations in one of the analyzed frames (frame 100 in this example)
# We specify a minimum localization separation of 28 pixels (2 x aoi_size).
import storm_analysis.multi_plane.psf_localizations as psfLocalizations
psfLocalizations.psfLocalizations("c1_zcal.hdf5", "map.map", frame = 100, aoi_size = 14)

In [ ]:
# Visual check of the selected localizations.
print("Channel 1 (frame 100)")
multiplane_measure_psf.overlayImage("c1_zcal.dax", "c1_zcal_c1_psf.hdf5", 100)

print("Channel 2 (frame 50)")
multiplane_measure_psf.overlayImage("c2_zcal.dax", "c1_zcal_c2_psf.hdf5", 50)

### Measure Z stacks for each channel.

In [ ]:
import storm_analysis.multi_plane.psf_zstack as psfZStack

psfZStack.psfZStack("c1_zcal.dax", "c1_zcal_c1_psf.hdf5", "c1_zstack", scmos_cal = "calib.npy", aoi_size = 12)
psfZStack.psfZStack("c2_zcal.dax", "c1_zcal_c2_psf.hdf5", "c2_zstack", scmos_cal = "calib.npy", aoi_size = 12)


Note: If you have linear drift in X/Y during PSF measurement you can correct for it at this stage using the `driftx` and `drifty` arguments to `psfZStack()`. One way to estimate the correct values to use is to analyze the entire calibration movie, then use `multi_plane.zstack_xydrift.xyDrift()`.

### Measure the PSFs for each channel.

If you are doing multicolor analysis at the same time then you'll want to use `normalize = True`. In multicolor analysis each camera would also be detecting different light wavelengths.

As with `Spliner` PSF measurement, you will need a text file containing the z-offset of each frame in the movie. This file contains two columns, the first is whether or not the data in this frame should be used (0 = No, 1 = Yes) and the second contains the z offset in microns. In this example this was automatically generated by the `makeSampleData()` function.

In [ ]:
# Load sample z_offsets.txt file. In this example every frame is valid.
import numpy
z_offsets = numpy.loadtxt("z_offset.txt")

print(z_offsets[:5,:])

In [ ]:
# PSF measurement
import storm_analysis.multi_plane.measure_psf as measurePSF

print("{0:.3f} pixel size (nm)".format(multiplane_measure_psf.pixel_size))
print("{0:.3f} spline z range (um)".format(multiplane_measure_psf.spline_z_range))
print()

measurePSF.measurePSF("c1_zstack.npy", 
                      "z_offset.txt", 
                      "c1_psf.psf", 
                      pixel_size = multiplane_measure_psf.pixel_size * 1.0e-3, # Pixel size is in microns.
                      refine = False, 
                      z_range = multiplane_measure_psf.spline_z_range, # Spline z range in microns. 
                      z_step = 0.050, 
                      normalize = False)

print()
measurePSF.measurePSF("c2_zstack.npy", 
                      "z_offset.txt", 
                      "c2_psf.psf", 
                      pixel_size = multiplane_measure_psf.pixel_size * 1.0e-3, # Pixel size is in microns.
                      refine = False, 
                      z_range = multiplane_measure_psf.spline_z_range, # Spline z range in microns. 
                      z_step = 0.050, 
                      normalize = False)

### Visualize the measured PSFs

In [ ]:
import storm_analysis.jupyter_examples.psf_images as psfImages

psfImages.psfImages("c1_psf.psf")
psfImages.psfImages("c2_psf.psf")


The next step is to convert the measured PSFs to splines. This is discussed in the `multiplane_psfs_to_splines` notebook.

You can also examine the PSFs with a tool like ImageJ. Tiff versions of the PSFs are also created, "c1_psf.tif" and "c2_psf.tif" in this example.